# Webscraping Project on Flight Price Prediction

In [211]:
# to import requried libraries
import selenium
import pandas as pd
import numpy as np
import time
import requests
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

In [212]:
# to get web drivers
s=Service(r"C:\Users\DELL\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(service=s)

In [213]:
# to get webpage of mentioned url
url = "https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=DEL&originCountry=IN&destination=BOM&destinationCountry=IN&flight_depart_date=20%2F07%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=901043949405"
driver.get(url)
time.sleep(3)

In [214]:
# to make a list of locations to scrap data
location_lst = ['Pune', 'Bangalore', 'Chennai','Hyderabad ','Goa ','Kolkata ','Jaipur ','Lucknow ']

In [215]:
# empty list to store data:
Airline = []
Source = []
Destination = []
Departure_Time = []
Arrival_Time = []
Duration = []
Meal=[] 
Total_Stops = []
Price = []
Location=[]

In [218]:
# Scraping the data from the mentioned url

# to get  webelement of source 
for x in location_lst:
    for y in location_lst:
        if x!=y:                 
            Source = driver.find_element(By.XPATH,"//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(2)
            Source.clear() 
            Source.send_keys(x)
            time.sleep(2)
            
            # to get webelement of destination
            Destination = driver.find_element(By.XPATH,"//div[@class='input-holder  bdr-btm pb-2']/input")
            Destination.click()
            Destination.clear()
            time.sleep(2)
            Destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element(By.XPATH,"/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            # Searching for flights again and clicking in search again button
            try:
                srch_btn = driver.find_element(By.XPATH,"//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(2)
            
            
            # to get web element of scroll to top button
            try:
                driver.find_element(By.XPATH,"//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
            
           # to get name of Airline
            try:
                for i in driver.find_elements(By.XPATH,"//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    Airline.append(i.text)
            except NoSuchElementException:
                pass
            
            # to get departure time of the flights
            try:
                for i in driver.find_elements(By.XPATH,"//div[@class='i-b pr']"):
                    Departure_Time.append(i.text)
            except NoSuchElementException:
                pass
            
            # to get arrival time of the flights 
            try:
                for i in driver.find_elements(By.XPATH,"//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    Arrival_Time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            
            # to get Duration of flights journey
            try:
                for i in driver.find_elements(By.XPATH,"//div[@class='stop-cont pl-13']/p"):
                    Duration.append(i.text)
            except NoSuchElementException:
                pass
            
            # to get number of stops the flights have between source and destination
            try:
                for i in driver.find_elements(By.XPATH,"//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    Total_Stops.append(i.text)
            except NoSuchElementException:
                pass
            
            # to get availability of meal in flights
            try:
                for i in driver.find_elements(By.XPATH,"//div[@class='features pull-right fs-12 flex']/div[1]"):
                    Meal.append(i.text)
            except NoSuchElementException:
                pass
            
            # to get Locations
            try:
                for i in driver.find_elements(By.XPATH,"//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    Location.append(i.text)
            except NoSuchElementException:
                pass
            
            # to get Prices of flight tickets
            try:
                for i in driver.find_elements(By.XPATH,"//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    Price.append(i.text)
            except NoSuchElementException:
                pass    

In [220]:
# Checking length of source and destination
len(Source), len(Destination)

(1425, 1425)

In [173]:
# Creating a dataframe for scraped data
data = list(zip(Airline,Departure_Time,Arrival_Time,Duration,Source,Destination,Meal,Total_Stops,Price))
df = pd.DataFrame(data, columns = ["Airline",
                                   "Departure_Time",
                                   "Arrival_Time",
                                   "Duration", 
                                   "Source",
                                   "Destination",
                                   "Meal_availability",
                                   "Total_Stops",
                                   "Price"])

In [174]:
df

,Airline,Departure_Time,Arrival_Time,Duration,Source,Destination,Meal_availability,Total_Stops,Price
0,Air India,17:55,22:10,4h 15m,New Delhi,Mumbai,Free Meal,1 Stop,"5,955"
1,Go First,17:05,21:10,4h 05m,New Delhi,Mumbai,eCash 250,1 Stop,"7,108"
2,Vistara,15:30,17:40,2h 10m,New Delhi,Mumbai,Free Meal,Non Stop,"8,578"
3,Vistara,16:55,19:05,2h 10m,New Delhi,Mumbai,Free Meal,Non Stop,"8,578"
4,Vistara,12:45,15:00,2h 15m,New Delhi,Mumbai,Free Meal,Non Stop,"8,578"
...,...,...,...,...,...,...,...,...,...
4567,Air Asia,18:45,21:40,2h 55m,New Delhi,Bangalore,Free Meal,Non Stop,"13,829"
4568,Vistara,19:50,22:40,2h 50m,New Delhi,Bangalore,Free Meal,Non Stop,"14,355"
4569,Vistara,13:20,19:15,29h 55m,New Delhi,Bangalore,Free Meal,2 Stop(s),"14,631"
4570,Vistara,18:40,15:55,21h 15m,New Delhi,Bangalore,Free Meal,1 Stop,"14,981"


In [221]:
# Creating a dataframe for scraped data
data = list(zip(Airline,Departure_Time,Arrival_Time,Duration,Source,Destination,Meal,Total_Stops,Price))
df3 = pd.DataFrame(data, columns = ["Airline",
                                   "Departure_Time",
                                   "Arrival_Time",
                                   "Duration", 
                                   "Source",
                                   "Destination",
                                   "Meal_availability",
                                   "Total_Stops",
                                   "Price"])

In [222]:
df3

,Airline,Departure_Time,Arrival_Time,Duration,Source,Destination,Meal_availability,Total_Stops,Price
0,Air Asia,19:15,20:45,1h 30m,Pune,Bangalore,eCash 250,Non Stop,"7,413"
1,IndiGo,22:25,23:55,1h 30m,Pune,Bangalore,eCash 250,Non Stop,"7,413"
2,IndiGo,19:05,20:40,1h 35m,Pune,Bangalore,eCash 250,Non Stop,"7,413"
3,Go First,19:40,21:15,1h 35m,Pune,Bangalore,eCash 250,Non Stop,"7,413"
4,Vistara,18:55,20:30,1h 35m,Pune,Bangalore,Free Meal,Non Stop,"7,938"
...,...,...,...,...,...,...,...,...,...
1420,Vistara,20:35,09:00,12h 25m,Pune,Bangalore,Free Meal,1 Stop,"22,953"
1421,Vistara,20:35,22:40,26h 05m,Pune,Bangalore,Free Meal,1 Stop,"23,688"
1422,Air India,21:55,18:20,20h 25m,Pune,Bangalore,Free Meal,2 Stop(s),"23,793"
1423,Air India,21:55,18:20,20h 25m,Pune,Bangalore,Free Meal,2 Stop(s),"23,793"


In [230]:
#to merge df1 and df3 in to single dataset
df1=df.append(df3, ignore_index=True)

In [231]:
df1

,Airline,Departure_Time,Arrival_Time,Duration,Source,Destination,Meal_availability,Total_Stops,Price
0,Air India,17:55,22:10,4h 15m,New Delhi,Mumbai,Free Meal,1 Stop,"5,955"
1,Go First,17:05,21:10,4h 05m,New Delhi,Mumbai,eCash 250,1 Stop,"7,108"
2,Vistara,15:30,17:40,2h 10m,New Delhi,Mumbai,Free Meal,Non Stop,"8,578"
3,Vistara,16:55,19:05,2h 10m,New Delhi,Mumbai,Free Meal,Non Stop,"8,578"
4,Vistara,12:45,15:00,2h 15m,New Delhi,Mumbai,Free Meal,Non Stop,"8,578"
...,...,...,...,...,...,...,...,...,...
5992,Vistara,20:35,09:00,12h 25m,Pune,Bangalore,Free Meal,1 Stop,"22,953"
5993,Vistara,20:35,22:40,26h 05m,Pune,Bangalore,Free Meal,1 Stop,"23,688"
5994,Air India,21:55,18:20,20h 25m,Pune,Bangalore,Free Meal,2 Stop(s),"23,793"
5995,Air India,21:55,18:20,20h 25m,Pune,Bangalore,Free Meal,2 Stop(s),"23,793"


In [236]:
#to save the dataframe in to a excel file 
df1.to_excel("Flight_Prices.xlsx")